<a href="https://colab.research.google.com/github/kartik727/neural-machine-translation/blob/master/Seq2Seq_with_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Seq2Seq with attention

## Dependencies
Primary library used for modelling and training - trax

## Data - Tensorflow Datasets (TFDS)
1. OPUS (`'opus'`)

In [ ]:
# instal trax

!pip install trax

In [29]:
import random
import numpy as np
import re
import nltk

import trax
from trax import layers as tl
from trax.fastmath import numpy as fastnp
from trax.supervised import training

In [3]:
import tensorflow_datasets as tfds

In [67]:
import tensorflow as tf

In [35]:
from collections import defaultdict

In [32]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
dataset_train = tfds.load('opus', split='train', batch_size=-1, shuffle_files=True)

In [11]:
ds_np = tfds.as_numpy(dataset_train)

In [14]:
ds_np['de'].shape

(1108752,)

In [15]:
# Utils Namespace

class Namespace:
    def __init__(self, **kwargs):
        self.update(**kwargs)

    def update(self, **kwargs):
        self.__dict__.update(kwargs)

In [ ]:
config_dict = {
    'vocab_size_en' = 20_000,
    'vocab_size_de' = 20_000
}



In [61]:
def preprocess_sentence(w):
  w = w.decode().lower().strip()

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [ ]:
def preprocess_data(data):
    return [preprocess_sentence(w) for w in data]

In [63]:
preprocessed_data_en, preprocessed_data_de = preprocess_data(ds_np['en']),preprocess_data(ds_np['de'])

In [ ]:
preprocessed_data_en

In [74]:
del ds_np

In [65]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [69]:
token_en = tokenize(preprocessed_data_en)

In [75]:
del preprocessed_data_en

In [76]:
token_de = tokenize(preprocessed_data_de)